# Physician Notetaker

Link to Assessment : https://sunset-parrot-38b.notion.site/Physician-Notetaker-1ac7fca3d41680849078deb25228b34f

## Task 1 : Medical NLP Summarization 

**Task:** Implement an NLP pipeline to **extract medical details** from the transcribed conversation.

### **📍 Deliverables:**

1. **Named Entity Recognition (NER):** Extract **Symptoms, Treatment, Diagnosis, Prognosis** using `spaCy` or `transformers`.
2. **Text Summarization:** Convert the transcript into a **structured medical report**.
3. **Keyword Extraction:** Identify **important medical phrases** (e.g., "whiplash injury," "physiotherapy sessions").

**📍 Sample Input (Raw Transcript):**

```
text
CopyEdit
Doctor: How are you feeling today?
Patient: I had a car accident. My neck and back hurt a lot for four weeks.
Doctor: Did you receive treatment?
Patient: Yes, I had ten physiotherapy sessions, and now I only have occasional back pain.

```

**📍 Expected Output (Structured Summary in JSON Format):**

```json
json
CopyEdit
{
  "Patient_Name": "Janet Jones",
  "Symptoms": ["Neck pain", "Back pain", "Head impact"],
  "Diagnosis": "Whiplash injury",
  "Treatment": ["10 physiotherapy sessions", "Painkillers"],
  "Current_Status": "Occasional backache",
  "Prognosis": "Full recovery expected within six months"
}

```

**📍 Questions:**

- How would you handle **ambiguous or missing medical data** in the transcript?
- What **pre-trained NLP models** would you use for medical summarization?



## Task 1 Using Spacy

In [1]:
# !pip uninstall spacy
# !pip uninstall en-core-web-sm
!pip install spacy==3.5.3
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.2 MB 12.7 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/12.2 MB 12.7 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/12.2 MB 12.7 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/12.2 MB 12.7 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/12.2 MB 1.5 MB/s eta 0:00:07
   ------ --------------------------------- 2.1/12.2 MB 1.8 MB/s eta 0:00:06
   ------ --------------------------------- 2.1/12.2 MB 1.8 MB/s eta 0:00:06
   ------ --------------------------------- 2.1/12.2 MB 1.8 MB/s eta 0:00:06
   ---------- ----------------------------- 3.1/12.2 MB 1.6 MB/s eta 0:00:06
   ------------ --------------------------- 3.9/12.2 MB 1.8 MB/s eta 0:00:05
   ------------- -------------------------- 4.2/12.2 MB 1.9 MB/s eta 0:00:05
 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-sci-md 0.5.3 requires spacy<3.7.0,>=3.6.1, but you have spacy 3.5.3 which is incompatible.
en-core-sci-sm 0.5.1 requires spacy<3.5.0,>=3.4.1, but you have spacy 3.5.3 which is incompatible.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.5.3 which is incompatible.
scispacy 0.5.5 requires spacy<3.8.0,>=3.7.0, but you have spacy 3.5.3 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable

2025-03-08 23:02:15.261530: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 23:02:17.300821: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 10.7 MB/s eta 0:00:02
     ------------- -------------------------- 4.5/12.8 MB 10.7 MB/s eta 0:00:01
     ------------------ --------------------- 5.8/12.8 MB 9.8 MB/s eta 0:00:01
     ------------------ --------------------- 6.0/12.8 MB 9.2 MB/s eta 0:00:01
     -------------------- ------------------- 6.6/12.8 MB 6.7 MB/s eta 0:00:01
     ----------------------- ---------------- 7.6/12.8 MB 5.9 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 5.8 MB/s eta 0:00:01
     ------------------------------- -------- 10.0/12.8 MB 5.9 MB/s eta 0:00:01
     ----------------------------------- ---- 11.3/12.8 MB 5.9 MB/s eta 0:00:01
     ------------------------------------- -- 12.1/12.8 MB 5.7 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 5.6 MB/s eta 0:00:01
     ---------------------------------------  12.6/1

In [2]:
import spacy

nlp = spacy.load('en_core_web_sm')

print(nlp) # Loading the model

In [3]:
def sanitize_conversation(text:str)->str:
    """
    Sanitizes and cleans the raw conversation text (removes * > symbols)
    """
    text = text.replace("*","")
    text = text.replace(">","")
    return text.strip()


with open("convo.txt","r") as f:
      text = f.read()
cleaned_text = sanitize_conversation(text)
print(cleaned_text)

# Save cleaned text to a text file.
with open("cleaned_convo.txt","w") as f:
    f.write(cleaned_text)
# Run the above snippet once to generate a neat conversation file.

doc = nlp(cleaned_text)
print(doc)

Physician: Good morning, Ms. Jones. How are you feeling today?
 
 
 Patient: Good morning, doctor. Iâ€™m doing better, but I still have some discomfort now and then.
 
 Physician: I understand you were in a car accident last September. Can you walk me through what happened?
 
 Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
 
 Physician: That sounds like a strong impact. Were you wearing your seatbelt?
 
 Patient: Yes, I always do.
 
 Physician: What did you feel immediately after the accident?
 
 Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
 
 Physician: Did you seek medical attention at that time?
 
 Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and 

In [9]:
# Lets try downloading pretrained medical usecase models

# Uncomment to run this / install on your system ----
#                                                    |

# !pip install scispacy 
!pip install "https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_scibert-0.5.1.tar.gz"
# en_core_sci_scibert is mainly trained for biomedical usecases, entities

Defaulting to user installation because normal site-packages is not writeable

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.0 requires typer>=0.9.0, but you have typer 0.7.0 which is incompatible.
embedchain 0.1.103 requires alembic<2.0.0,>=1.13.1, but you have alembic 1.8.1 which is incompatible.
embedchain 0.1.103 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.3 which is incompatible.
embedchain 0.1.103 requires tiktoken<0.6.0,>=0.5.2, but you have tiktoken 0.8.0 which is incompatible.
litellm 1.57.1 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.21 which is incompatible.
sentence-transformers 3.2.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.26.1 which is incompatible.
trl 0.8.1 requires transformers>=4.31.0, but you have transformers 4.26.1 which is incompatible.



     ---------------------------------------- 0.0/417.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/417.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/417.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/417.6 MB ? eta -:--:--
     ---------------------------------------- 0.3/417.6 MB ? eta -:--:--
     -------------------------------------- 0.5/417.6 MB 882.6 kB/s eta 0:07:53
     ---------------------------------------- 0.8/417.6 MB 1.0 MB/s eta 0:06:38
     ---------------------------------------- 1.8/417.6 MB 1.8 MB/s eta 0:03:48
     ---------------------------------------- 1.8/417.6 MB 1.8 MB/s eta 0:03:48
     ---------------------------------------- 3.9/417.6 MB 2.9 MB/s eta 0:02:23
      --------------------------------------- 5.5/417.6 MB 3.5 MB/s eta 0:01:58
      --------------------------------------- 6.6/417.6 MB 3.7 MB/s eta 0:01:53
      --------------------------------------- 7.6/417.6 MB 3.9 MB/s eta 0:

In [11]:
import scispacy
import spacy
# en_core_web_sm is not good for our use case, so lets use en_core_sci_sm here 
nlp = spacy.load("en_core_sci_scibert")
cleaned_text = """
Physician: Good morning, Ms. Jones. How are you feeling today?
 
 
 Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.
 
 Physician: I understand you were in a car accident last September. Can you walk me through what happened?
 
 Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.
 
 Physician: That sounds like a strong impact. Were you wearing your seatbelt?
 
 Patient: Yes, I always do.
 
 Physician: What did you feel immediately after the accident?
 
 Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.
 
 Physician: Did you seek medical attention at that time?
 
 Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.
 
 Physician: How did things progress after that?
 
 Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.
 
 Physician: That makes sense. Are you still experiencing pain now?
 
 Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.
 
 Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?
 
 Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.
 
 Physician: And how has this impacted your daily life? Work, hobbies, anything like that?
 
 Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.
 
 Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.
 
 [Physical Examination Conducted]
 
 Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.
 
 Patient: That’s a relief!
 
 Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.
 
 Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?
 
 Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.
 
 Patient: Thank you, doctor. I appreciate it.
 
 Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""
doc = nlp(cleaned_text)
print(doc)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Rizen3\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Rizen3\AppData\Local\Temp\ipykernel_12580\374138958.py", line 4, in <module>
    nlp = spacy.load("en_core_sci_scibert")
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rizen3\AppData\Roaming\Python\Python311\site-packages\spacy\__init__.py", line 54, in load
    return util.load_model(
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Rizen3\AppData\Roaming\Python\Python311\site-packages\spacy\util.py", line 442, in load_model
    def load_model_from_package(
                       ^^^^^^^^^^
  File "C:\Users\Rizen3\AppData\Roaming\Python\Python311\site-packages\spacy\util.py", line 478, in load_model_from_package
    exclude: Union[str, Iterable[str]] = _DEFAULT_EMPTY_PIPES,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

In [8]:
print(doc.ents)

(Physician, Good morning, Ms. Jones, Patient, Good morning, doctor, I, discomfort, car accident, happened, Patient, afternoon, I, Cheadle, Manchester, I, stop, traffic, nowhere, car, behind, car, Physician, sounds, impact, seatbelt, Patient, accident, Patient, I, realized I, I, pain, neck, right away, Physician, medical attention, Patient, Moss Bank Accident, Emergency, whiplash injury, X-rays, advice, sent, progress, Patient, weeks, neck, back pain, badâ€”I, sleeping, painkillers, regularly, improving, sessions, physiotherapy, stiffness, discomfort, sense, pain, Patient, Itâ€™s not constant, occasional backaches, Thatâ€™, hear, effects, anxiety, driving, difficulty, Patient, nervous driving, I havenâ€™t, emotional issues, accident, impacted, hobbies, Patient, week, I, routine, doing, anything, Thatâ€™s encouraging, ahead, physical examination, check, mobility, lingering pain, Physical Examination Conducted]
 
 Physician, Everything looks, neck, back, movement, thereâ€™s no tenderness,